**Install Dependencies**

In [2]:
!pip install praw
!pip install flask

**Use Flask to handle incoming requests from redirect URI from Reddit**

In [4]:
from flask import Flask, request

app = Flask(__name__)

@app.route('/reddit_callback')
def reddit_callback():
    # Retrieve the authorization code or access token from the URL parameters
    authorization_code = request.args.get('code')
    # Do something with the authorization code, such as exchanging it for an access token
    # Or, store it for later use
    return "Callback received successfully"

if __name__ == '__main__':
    app.run(host='localhost', port=8088)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://localhost:8088
INFO:werkzeug:Press CTRL+C to quit


**Gather data from Reddit using PRAW**

In [10]:
# Step 1: pip install praw, pip install flask
# Step 2: Go to the Reddit apps page (https://www.reddit.com/prefs/apps) while logged into your Reddit account.
# Step 3: Click on “are you a developer? create an app...” or “create another app”.
#         Fill out the form:
#         Name your application.
#         Select the “script” option.
#         Note down the client ID (found underneath the app name), user agent (your name found above the app name), and the client secret.
# Step 4: Input them below, and then run the cell

import praw
import pandas as pd

reddit = praw.Reddit(
    client_id='YOUR_CLIENT_ID',
    client_secret='YOUR_CLIENT_SECRET',
    user_agent='YOUR_USER_AGENT',
    check_for_async=False
)

# Step 4: Define the subreddit
subreddit = reddit.subreddit('changemyview')


# Step 5: Save the posts
# Create lists to store data
titles = []
bodies = []
urls = []
comments_list = []

# Iterate over the posts
for submission in subreddit.new(limit=10):  # Adjust the limit as needed
    submission.comments.replace_more(limit=0)  # Remove MoreComments
    comments = submission.comments.list()[:10]  # Get top 10 comments

    # Append data to lists
    titles.append(submission.title)
    bodies.append(submission.selftext)
    urls.append(submission.url)
    comments_list.append([comment.body for comment in comments])

# Creating a DataFrame
df = pd.DataFrame({
    'Title': titles,
    'Body': bodies,
    'URL': urls,
    'Top Comments': comments_list
})

print(df)
# Saving the df
df.to_csv('Reddit.csv', index=False)

                                               Title  \
0  CMV: Older coins for collecting should be clea...   
1  CMV: Saying Boomer had it easier is agreeing w...   
2  CMV: A dead bedroom alone is not reason enough...   
3                           META: Fresh Topic Friday   
4  CMV: The only way to curb the fentanyl epidemi...   
5  CMV: Military *femboy outfits are stupid (and ...   
6  CMV: "Virginity is a social construct" isn't a...   
7  CMV: There is no valid reason for tipping in a...   
8  CMV: consumers are ultimately responsible for ...   
9           CMV: Libertarianism is liberal extremism   

                                                Body  \
0  The cardinal sin of numismatics: cleaning coin...   
1  always wondered, on the one hand everytime som...   
2  It's no secret that when people ask for relati...   
3  [Every Friday](https://www.reddit.com/r/change...   
4  Especially in the US. we have draconian drug l...   
5  \*or any military outfit with thigh highs/sk